<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# Context Managers

In Python, a context manager is an object that allows you to control the context in which to run code. You can define how the context is created, and then close or "tear down" the context when you are finished.

<a id='resources'></a>
## Managing resources in Python

In any programming language, the usage of resources like files and databases is very common. But it is important to release these resources after usage. Otherwise we can cause memomry issues and risk other unintended side effects.

For example, let's open the following text file and read it in as a string to interact with.

In [ ]:
my_file = open('data/example.txt')

In [ ]:
text = my_file.read()

In [ ]:
print(text)

In [ ]:
len(text)

Notice at the moment, the file is still *open*.

In [ ]:
my_file.closed

It can cause issues if too many files are open as they take up space in memory.

Uncomment and run the cell below to demonstrate this.

In [ ]:
# file_descriptors = []
# for x in range(100000):
#     file_descriptors.append(open('data/example.txt', 'r'))

We get an error message saying that too many files are open. 

*Restart the kernel and continue.*

To avoid situations like above, when we have finished with a file we should close it.

In [ ]:
my_file = open('data/example.txt')

In [ ]:
my_file.close()

In [ ]:
my_file.closed

However, it would be very helpful if user have a mechanism for the automatic setup and teardown of resources.

In fact, as the `open()` function is a **context manager**, it can facilitate the proper handling of resources.

The most common way to do so is by using the `with` keyword. As shown below, it allows us to interact with our file by creating creates a *runtime context*, which is then closed afterwards.

In [ ]:
with open('data/example.txt') as my_file:
    text = my_file.read()
    length = len(text)
    print(text)
    print(length)

In [ ]:
my_file.closed

In [ ]:
print(f'The file is {length} characters long and the first word is {text.split()[0]}')

<a id='caterers'></a>
## Caterers are Context Managers


<img src='images/party.jpeg' width=500px>

Imagine you are hosting a fancy party. You may get caterers to help with the food and refreshments. 

In this situation, the caterers are analogous to the work that context managers do.

|Context Manager|Caterers|
|:---|:---|
|Set up a context|Set up the tables/prepare the food/drinks}
|Run your code|Leave you to party|
|Tear down the context|Clean up the mess|

<a id='use'></a>

## Using a context manager:

To use a context manager you open the context with the keyword `with`. Any code written in the indented block will run in the context.

```python
with <context-manager>(<args>) as <variable-name>:
    # your code here
    # this code is running 'inside the context'
        
# This code runs after the context is removed
```

---

<a id='ex-use'></a>

## <mark>Exercise: Practice using context managers</mark>

For exercise 1 you will need the following information about the data to use:

|File Name|Full Book Name|
|---|---|
|`data/alice.txt`|Alice's Adventures in Wonderland|
|`data/frankenstein.txt`|Frankenstein; or, The Modern Prometheus|
|`data/pride.txt`|Pride and Prejudice|


#### **Exercise 1:** Count how many times Lewis Caroll uses the word rabbit in the first chapter of Alice's Adventures in Wonderland

- Open `"data/alice.txt"` and assign the file to `file`.
- Using `file.read()` assign a new variable text with the contents of `alice.txt`.
- Use the `str.count()` method to count the number of times the word `rabbit` appears.

In [ ]:
# %load answers/ex-use1.py

## Building your own context managers

There are two ways to build a context manager. With either a `OOP-based` or **`generator-based`** approach. 

### OOP-based context managers

To define custom context managers we need to create a class that implements `__enter__()` and `__exit__()`.

- You can also define `__init__()` to specify arguments that the context manager can take.
- `__exit__()` needs to accept a reference to `self`, the type of exception it might throw, the exception itself and a traceback object as arguments.

In [ ]:
import sqlite3
import pandas as pd

class DBConnection:

    def __init__(self, db_name):
        self.db = db_name

    def __enter__(self):
        self.conn = sqlite3.connect(self.db)
        return self.conn

    def __exit__(self, exc_class, exc, traceback):
        self.conn.close()
        
        
with DBConnection('SQLDatabase.db') as db:
     trends = pd.read_sql('''SELECT * FROM programming_trends''', db)

trends.head()

#### **Exercise 2:** Create a context manager called `InDir` that allows you to run code form a different directory than the current working directory.

In [ ]:
!pwd

In [ ]:
# This code should work unedited
# with InDir('../../'):
#     notebook_files = os.listdir('notebooks')
    
# notebook_files

In [ ]:
!pwd # Should not have changed!

In [ ]:
# %load answers/ex-build1.py

#### **Exercise 3:** Create a context manager called `Timer` that times the execution time of code in its body.

- You can also add an extra argument that allows the user to add a description to the log of the execution time.

In [ ]:
# %load answers/ex-build2.py

### Generator-based context managers

Instead of creating context managers by designing classes, it's usually more ergonomic and idiomatic to use a generator function.

You can decorate any generator with the `@contextlib.contextmanager` decorator. The code before the `yield` statemt will act as the context set-up. The `yield` statement can return a handle to the object created in the context, and any code after `yield` will act as teardowm.

The quivalent to the previous database context manager would be:

In [ ]:
import contextlib
import sqlite3
import pandas as pd

@contextlib.contextmanager
def my_database(db_name):
    
    conn = sqlite3.connect(db_name)
    
    yield conn
    
    conn.close()
    

with my_database('SQLDatabase.db') as db:
    trends = pd.read_sql('''SELECT * FROM programming_trends''', db)
    
trends.head()

Python's `sqlite` package actually comes with it's own context manager, great! So we can use that instead. The above demonstrates the flow of using a context manager while connecting to SQL.

In [ ]:
import sqlite3

with sqlite3.connect('SQLDatabase.db') as conn:
    query = '''SELECT * FROM programming_trends'''
    results = conn.execute(query).fetchall()

results

**Caveat**: Often you would want to include a `try` (`except`) and `finally` within the function to ensure you are able to handle any connection errors you might have.

#### **Exercise 4:** Recreate the context managers from exercises 2 and 3 with generators

In [ ]:
# %load answers/ex-convert2.py